# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, SimpleRNN
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GroupKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d

# Define X and Y

In [2]:
# Load the saved features from the .npz file
data = np.load('combined_freq.npz') 

# Access the concatenated features from the .npz file
X_freq = data['X_frequency']

# Close the loaded file
data.close()

#print the merged array
X_freq

array([[[ 1.46584672e-13,  6.45390788e-21,  1.15009007e-25, ...,
          1.04873571e+00,  1.75910640e+00,  9.32599360e-06],
        [ 8.03138316e-14,  1.93266859e-21,  6.30700474e-26, ...,
          2.39037981e-01,  6.38193078e-01, -6.60421011e-06],
        [ 1.71122109e-13,  2.75453640e-23,  2.63616577e-25, ...,
          3.38987332e-01,  5.25518596e-01, -5.34106217e-06],
        ...,
        [ 1.52121787e-13,  8.47677259e-23,  2.30035066e-25, ...,
          2.90477535e-01,  3.20355828e-01, -9.29930152e-07],
        [ 1.50910431e-13,  4.19486197e-22,  2.48974250e-25, ...,
          5.94779033e-02,  2.76903208e-01,  1.20278818e-07],
        [ 4.84596455e-13,  4.15998616e-21,  2.19098742e-24, ...,
          2.88563512e-01,  8.60427949e-01,  3.48740780e-06]],

       [[ 2.12847299e-12,  7.43448561e-21,  5.54611187e-23, ...,
          6.50319496e-02,  1.25496815e-01, -8.67524675e-06],
        [ 3.88678177e-13,  3.17041833e-21,  1.55235289e-24, ...,
          1.08033075e-01,  3.88190787e

In [3]:
data = np.load('combined_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('combined_groups.npz')

group = data['group']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

# Reshape Array

In [5]:
groups_array = np.hstack(group)

In [6]:
# Check the shape of the concatenated arrays
print(X_freq.shape, Y.shape, groups_array.shape)

(73519, 20, 20) (73519,) (73519,)


In [7]:
# # Move the channel axis to the last dimension in the EEG data array
# epochs_array = np.moveaxis(X_merged_features, 1, 2)

# # Check the shape of the modified EEG data array
# epochs_array.shape

# Develop the Autoregression RNN Model

In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Attention
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np

# Define the improved CNN-LSTM hybrid model
def cnn_lstm_model(input_shape):

    learning_rate = 0.001
    dropout_rate = 0.3
    # Create a Sequential model
    model = Sequential()
    
    # Add Convolutional layers
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu',  padding='same', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    
    model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    # model.add(Dropout(dropout_rate))
    
    # Add Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    # model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(256)))
    # model.add(Dropout(dropout_rate))

    
    # Add Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer=Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef

# Define the evaluate_model function
def evaluate_model(model, features, labels):
    # Predict labels
    y_pred = model.predict(features)
    y_pred = np.round(y_pred).astype(int)
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, y_pred)
    precision = precision_score(labels, y_pred)
    recall = recall_score(labels, y_pred)
    f1 = f1_score(labels, y_pred)
    roc_auc = roc_auc_score(labels, y_pred)
    
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(labels, y_pred).ravel()
    specificity = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    
    # Calculate kappa and MCC
    total = tp + tn + fp + fn
    observed_accuracy = (tp + tn) / total
    expected_accuracy = ((tp + fp) * (tp + fn) + (tn + fp) * (tn + fn)) / (total ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return accuracy, precision, recall, f1, roc_auc, specificity, sensitivity, kappa, mcc

# Train the CNN LSTM Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import Adam
import os
import csv

# Define lists to store evaluation metrics for each fold
metrics_list = []

# Initialize confusion matrix sums
total_conf_matrix = [[0, 0], [0, 0]]

# Define the number of folds for cross-validation
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a directory for TensorBoard logs
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_freq, Y), 1):
    print(f"Fold {fold}")

    # Split the data into training and validation sets
    X_train, X_val = X_freq[train_index], X_freq[val_index]
    y_train, y_val = Y[train_index], Y[val_index]

    # Standardize features using StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_val_scaled = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)

    # Reshape the data for RNN input (samples, timesteps, features)
    timesteps = X_train_scaled.shape[1]
    features = X_train_scaled.shape[2]
    X_train_rnn = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, features))
    X_val_rnn = X_val_scaled.reshape((X_val_scaled.shape[0], timesteps, features))

    # Create an instance of the autoregression RNN model
    model = cnn_lstm_model((timesteps, features))

     # Set up TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=os.path.join(log_dir, f'fold_{fold}'))

    # Set up callbacks
    run_name = f"epoch_{fold}"
    checkpoint_filepath = f'{run_name}.h5'
    checkpoint_callback = ModelCheckpoint(
        checkpoint_filepath,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max'
    )

    csv_logger = CSVLogger(f'{run_name}.csv', append=True, separator=';')
    
    # Train the autoregression RNN model
    history = model.fit(X_train_rnn, y_train, epochs=200, batch_size=256, validation_data=(X_val_rnn, y_val), verbose=1, callbacks=[tensorboard_callback, checkpoint_callback, csv_logger])


    # Load the best model before evaluation
    best_model = load_model(checkpoint_filepath)

    # Evaluate client model
    accuracy, precision, recall, f1, sensitivity, specificity, roc_auc, kappa, mcc = evaluate_model(best_model, X_val_rnn, y_val)
    metrics_list.append({
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1': f1,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'ROC_AUC': roc_auc,
        'Kappa': kappa,
        'MCC': mcc
    })

Fold 1
Epoch 1/200
259/259 [==============================] - 13s 26ms/step - loss: 0.3638 - accuracy: 0.8332 - val_loss: 0.3157 - val_accuracy: 0.8630
Epoch 2/200
259/259 [==============================] - 5s 20ms/step - loss: 0.3002 - accuracy: 0.8648 - val_loss: 0.2750 - val_accuracy: 0.8766
Epoch 3/200
259/259 [==============================] - 5s 20ms/step - loss: 0.2764 - accuracy: 0.8774 - val_loss: 0.2522 - val_accuracy: 0.8855
Epoch 4/200
259/259 [==============================] - 5s 20ms/step - loss: 0.2571 - accuracy: 0.8876 - val_loss: 0.2466 - val_accuracy: 0.8969
Epoch 5/200
259/259 [==============================] - 5s 20ms/step - loss: 0.2446 - accuracy: 0.8936 - val_loss: 0.2408 - val_accuracy: 0.8998
Epoch 6/200
259/259 [==============================] - 5s 20ms/step - loss: 0.2343 - accuracy: 0.8984 - val_loss: 0.2307 - val_accuracy: 0.8983
Epoch 7/200
259/259 [==============================] - 5s 21ms/step - loss: 0.2248 - accuracy: 0.9030 - val_loss: 0.2211 - val_a

In [11]:
import pandas as pd
metrics_df = pd.DataFrame(metrics_list)

In [12]:
metrics_df.round(4)

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9650,0.9642,0.9704,0.9673,0.9647,0.9589,0.9704,0.9298,0.9298
1,0.9646,0.9678,0.9658,0.9668,0.9646,0.9633,0.9658,0.9290,0.9290
2,0.9610,0.9549,0.9727,0.9637,0.9601,0.9476,0.9727,0.9215,0.9217
3,0.9582,0.9544,0.9678,0.9611,0.9576,0.9473,0.9678,0.9160,0.9161
4,0.9574,0.9571,0.9632,0.9602,0.9570,0.9508,0.9632,0.9144,0.9145
5,0.9616,0.9664,0.9615,0.9639,0.9617,0.9619,0.9615,0.9230,0.9230
6,0.9591,0.9584,0.9650,0.9617,0.9586,0.9522,0.9650,0.9177,0.9177
7,0.9614,0.9638,0.9638,0.9638,0.9612,0.9586,0.9638,0.9224,0.9224
8,0.9600,0.9578,0.9676,0.9627,0.9595,0.9514,0.9676,0.9196,0.9197
9,0.9626,0.9627,0.9673,0.9650,0.9623,0.9572,0.9673,0.9248,0.9248


In [13]:
metrics_df.round(4).to_csv('FREQ_COMBINED_CNN_LSTM.csv', index = False)

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
result = pd.read_csv("FREQ_COMBINED_CNN_LSTM.csv")
result

,Accuracy,Precision,Recall,F1,Sensitivity,Specificity,ROC_AUC,Kappa,MCC
0,0.9650,0.9642,0.9704,0.9673,0.9647,0.9589,0.9704,0.9298,0.9298
1,0.9646,0.9678,0.9658,0.9668,0.9646,0.9633,0.9658,0.9290,0.9290
2,0.9610,0.9549,0.9727,0.9637,0.9601,0.9476,0.9727,0.9215,0.9217
3,0.9582,0.9544,0.9678,0.9611,0.9576,0.9473,0.9678,0.9160,0.9161
4,0.9574,0.9571,0.9632,0.9602,0.9570,0.9508,0.9632,0.9144,0.9145
5,0.9616,0.9664,0.9615,0.9639,0.9617,0.9619,0.9615,0.9230,0.9230
6,0.9591,0.9584,0.9650,0.9617,0.9586,0.9522,0.9650,0.9177,0.9177
7,0.9614,0.9638,0.9638,0.9638,0.9612,0.9586,0.9638,0.9224,0.9224
8,0.9600,0.9578,0.9676,0.9627,0.9595,0.9514,0.9676,0.9196,0.9197
9,0.9626,0.9627,0.9673,0.9650,0.9623,0.9572,0.9673,0.9248,0.9248


In [16]:
(result.mean()*100).round(2)

Accuracy       96.11
Precision      96.08
Recall         96.65
F1             96.36
Sensitivity    96.07
Specificity    95.49
ROC_AUC        96.65
Kappa          92.18
MCC            92.19
dtype: float64

In [18]:
# import csv
# for i in range (1, 11):
#     df = pd.read_csv(f'epoch_{i}.csv')
#     [col] = df.columns
#     split_data = df[col].str.split(';')
#     csv_file = (f'epoch_{i}.csv')
#     # Prepare CSV file and write headers
#     with open(csv_file, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Epoch', 'Training Accuracy', 'Training Loss', 'Validation Accuracy', 'Validation Loss'])
#     for j in range (0, 200):
#         with open(csv_file, mode='a', newline='') as file:
#             writer = csv.writer(file)
#             writer.writerow([split_data[j][0], split_data[j][1], split_data[j][2], split_data[j][3], split_data[j][4]])